In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch

In [ ]:
evo_fwd = np.load("/scratch/aa11803/d3pm/save_alphas/evodiff_fwd.npy")
evo_bwd = np.load("/scratch/aa11803/d3pm/save_alphas/evodiff_bwd.npy")
tau_fwd = np.load("/scratch/aa11803/d3pm/save_alphas/tauldr_fwd_nst32768.npy")
tau_bwd = np.load("/scratch/aa11803/d3pm/save_alphas/tauldr_rr_hist_nst32768_nsa100.npy")[::-1]

In [ ]:
lw = 2

def moving_average(x, window=1):
    return torch.nn.functional.avg_pool1d(x.unsqueeze(0), window, stride=1, padding=window//2).squeeze(0)

w = 1
bwd = evo_bwd * len(evo_bwd)
fwd = evo_fwd * len(evo_bwd)
ts = np.linspace(0.0, 1, len(evo_bwd))


fig, ax = plt.subplots(1, 2, figsize=[14, 2.7])

ax[0].plot(ts, 0*ts, color='black', ls='--')
ax[0].plot(ts, moving_average(torch.tensor(bwd - fwd), w),
         lw=lw, color='black')
ax[0].set_ylabel("Rate error", fontsize=20)
ax[0].set_xlabel("t", fontsize=20, labelpad=xlp)
ax[0].set_xticks([0, 1], [0, 1],fontsize=15)
ax[0].tick_params(axis='both', which='major', labelsize=20)
ax[0].set_xlim(0, 1)
ax[0].set_ylim(-0.85, 0.1)


w = 2 * (int(32768*0.003) // 2) + 1
min_t = 0.01
ts = np.linspace(0.001, 1, len(tau_bwd))
bwd = tau_bwd[ts > min_t]
fwd = tau_fwd[ts > min_t]
ts = ts[ts > min_t]

ax[1].plot(ts, 0*ts, color='black', ls='--')
ax[1].plot(ts, moving_average(torch.tensor(bwd - fwd), w),
         lw=lw, color='black')
ax[1].set_ylabel("Rate error", fontsize=20)
ax[1].set_xlabel("t", fontsize=20, labelpad=xlp)
ax[1].set_xticks([0, 1], [0, 1],fontsize=15)
ax[1].tick_params(axis='both', which='major', labelsize=20)
ax[1].set_xlim(0, 1)
ax[1].set_ylim(-0.6, 6.5)
plt.tight_layout()

plt.savefig("figures/divergence.pdf")